I have played PUBG a little more than few times and won a few as well. Like every other PvP action game we all tend to use some strategies to beat the game and other players. Here are a few which I tend to use in a match
    1.   **Loot** : As soon as I land there are inventory I wish to find, loot or win from the supply-drop. Good weapons can be your best friend (of course if you are shooting the ground)
    2.  **Walk rather than ride** : I prefer to travel by foot rather than use a vehicle. Main reasons being it alerts other players because of the noise. And also you are easily spotted and can be killed by exploding the vehicle by spraying bullets. This works best in Solo mode. I use slightly different tactics in duo and squad matches. And avoid swimming as much as possible.
    3.   **Camping** : Try to get to the center of the third radiation barrier as it turns out to be the final battlefied most of the time. (I have no basis to prove this)
    4.   Use **boost** and **heal** every time I get a chance.
    5.   **Landing** : Land as far as possible from the plane's linear path.
    6.   **Skills , skills, skills** - kills, headshots

With my own experience playing the game I consider headshots, walk/ride distance, weapon pick ups, headshotKills, boosts and heals are the factors that majorly influence the win percentage in a game. And ultimately help us predict the result of winPlacePerc.
    


### Objective: To support game experience with analysis on the data. Also to find insights that are not readily visible on first look of the game data. 
The steps I plan to proceed using are as follows: 
    1. Exploratory Data Analysis
    2. Preprocessing
    3. Prediction Model


    (Note : The Notebook might be incomplete and will add code along the way. This is my first time sharing my work in kernel. Any improvements and suggestions are welcome.)

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from scipy import stats
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
train_data = pd.read_csv('../input/train.csv')
test_data = pd.read_csv('../input/test.csv')

In [ ]:
train_data.head()

In [ ]:
train_data.tail()

In [ ]:
train_data.shape

In [ ]:
test_data.shape

In [ ]:
train_data.info()

In [ ]:
#Check for missing data
train_data.isna().sum()
#train_data.isnull().sum().sum()

## **Some information about player kills**

In [ ]:
train_data['kills'].describe()

Average kills or number of kills per player : 0.9345 
Almost 75% of the players have atleast 1 kill.
Maximum kills by a player in a match : 60   (...hmmm...looks fishy for a normal match. Probably in a custom mode) lets check the player stats below


In [ ]:
train_data[train_data['kills']==60]

In [ ]:
train_data[train_data['matchId']==5518]

Interesting. Match(Id : 5518) with max kill of 60 by a player seems to be a custom match with total kills more than 99 kills(max anyone can kill in a normal match without killing themselves)

In [ ]:
#sns.set()
sns.distplot(train_data['kills'],bins=40,kde=True)
plt.figure(figsize=(20,20))
plt.show()

In [ ]:
[x for x in train_data['kills'].quantile() if train_data['kills'].quantile(x)

*I need help getting the percentile of players with more than 1 kill.*

### **Total number of independent groups in training data : **

In [ ]:
train_data['groupId'].nunique()

Do Solo and custom matches have groupIds? 

**Lets find out**

In [ ]:
train_data[train_data['DBNOs']==0]

Realized zero DBNOs and assists doesnt necessarily mean its a solo match.

In [ ]:
train_data[train_data['winPlacePerc']==1].groupby(train_data['groupId']).tail()

In [ ]:
train_data[train_data['winPlacePerc']==1].groupby(train_data['Id']).tail()

In [ ]:
train_data.groupby(train_data['groupId'])['winPlacePerc'].value_counts().max()

In [ ]:
train_data['teamKills'].value_counts()

In [ ]:
train_data['kills'].value_counts()

In [ ]:
sns.jointplot(x='kills',y='winPlacePerc',data= train_data)

In [ ]:
sns.pairplot(train_data,x_vars=['kills','teamKills'],y_vars=['winPlacePerc'])
plt.figure(figsize=(15,15))

In [ ]:
sns.jointplot(x='winPlacePerc',y='boosts',data= train_data,height=10, ratio=3,color = 'orange')

In [ ]:
sns.jointplot(x='winPlacePerc',y='heals',data= train_data,height=10, ratio=3, color = 'green')

In [ ]:
sns.jointplot(x='winPlacePerc',y='headshotKills',data= train_data,height=10, ratio=3, color = 'red')

sns.set()
cols = ['winPlacePerc', 'walkDistance', 'boosts', 'weaponsAcquired', 'damageDealt', 'killPlace']
sns.pairplot(train_data[cols])
plt.show()

## **Preprocessing**

Before diving into modeling I think we can reduce the number of variables influencing the target('winPlacePerc'). We can achieve this by using Principal component analysis. But first the data has to be scaled.

(note: imports are done above)

In [ ]:
features = list(train_data.columns)
x = train_data.iloc[:,:25].values
y = train_data.loc[:,'winPlacePerc']
scaler = StandardScaler() 
x = scaler.fit_transform(x)

In [ ]:
pca = PCA(n_components = 25)
principalComp = pca.fit_transform(x)
principalDf = pd.DataFrame(data = principalComp)

In [ ]:
print(principalDf)

Instantiate an instance to reduce the dimentionality of the dataset with components with 95% variance

In [ ]:
pca= PCA(.95)

In [ ]:
pca.fit(x)


Display total number of components and array of components 

In [ ]:
pca.n_components_

In [ ]:
pca.components_

To display an array with information contained by all the components

In [ ]:
pca.explained_variance_ratio_

In [ ]:
plt.semilogy(pca.explained_variance_ratio_,'--o')

In [ ]:
pd.DataFrame(pca.components_,columns =x.columns)

In [ ]:
import numpy as np

In [ ]:
sns.heatmap(np.log(pca.inverse_transform(np.eye(x.shape[1]))), cmap="hot", cbar=False)
